In [1]:
import os
import numpy as np
import cv2
from torchvision import models
DATAS_UP_PATH='./output_up_camera/'
DATAS_DOWN_PATH='./output_down_camera/'

In [2]:
up_list = os.listdir(DATAS_UP_PATH)
donw_list = os.listdir(DATAS_DOWN_PATH)

In [3]:
# follow / to_position / line_num / loaded /// position_bool / people / x / y + filename
dataset=list()
for i in range(len(up_list)):
    if up_list[i][-4:] != '.jpg':continue
    follow, to_position, line_num, loaded, position_bool, people = up_list[i].split('_')[0]
    x = up_list[i].split('_')[1]
    y = up_list[i].split('_')[2]
    follow, to_position, line_num, loaded, position_bool, people = float(follow), float(to_position), float(line_num), float(loaded), float(position_bool), float(people)
    x, y = float(x), float(y)
    img_up = cv2.imread(DATAS_UP_PATH+up_list[0]).reshape(3,224,224)
    img_down = cv2.imread(DATAS_DOWN_PATH+up_list[0]).reshape(3,224,224)
    input_data = (follow, to_position, line_num, loaded, img_up, img_down)
    input_label = (position_bool, people, x, y)
    dataset.append((input_data, input_label))


In [4]:
import torch
BATCH_SIZE = 32
TEST_RATE = 0.2
TEST_SET_NUM = int(len(dataset)*TEST_RATE)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - TEST_SET_NUM, TEST_SET_NUM])

In [5]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

In [17]:
class AutoDrive(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.output_classes=4 # position_bool, people, x, y
        self.unit_output = 100
        self.model_up = self.making_transfer_model()
        self.model_down = self.making_transfer_model()
        self.determine_model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.5, inplace=True),
            torch.nn.Linear(in_features=204, out_features=self.output_classes, bias=True),
            torch.nn.Softmax()
        )
    
    def making_transfer_model(self):
        model = models.efficientnet_b7(pretrained=True)
        model.classifier=torch.nn.Sequential(
            torch.nn.Dropout(p=0.5, inplace=True),
            torch.nn.Linear(in_features=2560, out_features=self.unit_output, bias=True),
            torch.nn.Mish(),
        )
        return model

    def forward(self, follow, to_position, line_num, loaded, up_img, down_img): # follow / to_position / line_num / loaded
        # up_img = torch.from_numpy(up_img).float()
        # down_img = torch.from_numpy(down_img).float()
        x_up = self.model_up(up_img)
        print("check")
        x_down = self.model_up(down_img)
        x = torch.cat([follow,to_position, line_num, loaded, x_up, x_down], dim=-1)
        x = self.determine_model(x)
        return x


In [18]:
auto = AutoDrive()

In [ ]:
for (follow, to_position, line_num, loaded, up_img, down_img),(position_bool, people, x, y) in train_loader:
    print(up_img.shape)
    out = auto(follow, to_position, line_num, loaded, up_img.float(), down_img.float())
    print(out)
    break